In [1]:
# script to compile calculations from new and old workflow

In [2]:
import os
import re
import glob

import rmgpy.chemkin
import rmgpy.data.kinetics
import rmgpy.data.thermo

import numpy as np
# import importlib
# importlib.reload(rmgpy.data.kinetics)

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [3]:
# load the 1week model and uncertainty
# get the mapping from cantera base back to RMG
# load the sensitive parameters
# chemkin = '/home/moon/autoscience/reaction_calculator/delay_uncertainty/base_rmg_1week/chem_annotated.inp'
chemkin = '/work/westgroup/harris.se/autoscience/reaction_calculator/delay_uncertainty/base_rmg_1week/chem_annotated.inp'
working_dir = os.path.dirname(chemkin)
transport = os.path.join(working_dir, 'tran.dat')
species_dict = os.path.join(working_dir, 'species_dictionary.txt')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin, dictionary_path=species_dict, transport_path=transport, use_chemkin_names=True)
cantera_file = os.path.join(working_dir, 'chem_annotated.cti')
# base_gas = ct.Solution(cantera_file)
# perturbed_cti_path = os.path.join(working_dir, 'perturbed.cti')
# perturbed_gas = ct.Solution(perturbed_cti_path)


rxn_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_reaction_uncertainty.npy'
sp_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_species_uncertainty.npy'

rmg_rxn_uncertainty = np.load(rxn_uncertainty_file)
rmg_sp_uncertainty = np.load(sp_uncertainty_file)


# Add thermo

In [ ]:
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
thermo_libs = glob.glob(os.path.join(DFT_DIR, 'thermo', 'species*', 'arkane', 'RMG_libraries'))
print(f'{len(thermo_libs)} thermo libs')


# Load the Arkane thermo
entries = []
for i, lib_path in enumerate(thermo_libs):
    matches = re.search('species_([0-9]{4})', lib_path)
    species_index = int(matches[1])
    ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
    ark_thermo_database.load_libraries(
        lib_path,
    )

    for key in ark_thermo_database.libraries['thermo'].entries.keys():
        entry = ark_thermo_database.libraries['thermo'].entries[key]
        entry.index = species_index
        entry.label = entry.item.smiles
        entries.append(entry)
print(f'{len(entries)} entries')

In [ ]:
# also load new thermo:
DFT_DIR = "/work/westgroup/harris.se/autoscience/reaction_calculator/dft/"
thermo_libs = glob.glob(os.path.join(DFT_DIR, 'thermo', 'species*', 'arkane', 'RMG_libraries'))
print(f'{len(thermo_libs)} thermo libs')

indices = [e.index for e in entries]
# Load the Arkane thermo
# entries = []
for i, lib_path in enumerate(thermo_libs):
    matches = re.search('species_([0-9]{4})', lib_path)
    species_index = int(matches[1])
    if species_index in indices:
#         print('skipping', i)
        continue
    else:
        print('adding', species_index)
    
    ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
    ark_thermo_database.load_libraries(
        lib_path,
    )

    for key in ark_thermo_database.libraries['thermo'].entries.keys():
        entry = ark_thermo_database.libraries['thermo'].entries[key]
        entry.index = species_index
        entry.label = entry.item.smiles
        entries.append(entry)
print(f'{len(entries)} entries')

In [ ]:
def include_species(mol):
    for i in range(0, len(species_list)):
        species_list[i].generate_resonance_structures()
        for j in range(len(species_list[i].molecule)):
            if species_list[i].molecule[j].is_isomorphic(mol):
                if rmg_sp_uncertainty[i] >= 3.0:
                    return True
    return False

In [ ]:
# count = 0
# for j, entry in enumerate(entries):
#     if include_species(entry.item):
#         print(j, entry)
#         count += 1

# # include_species(entries[0].item)

In [ ]:
# compile it all into a single database and a single library which I'll call harris_butane
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.libraries['thermo'] = rmgpy.data.thermo.ThermoLibrary()
ark_thermo_database.libraries['thermo'].label = 'harris_butane'
ark_thermo_database.libraries['thermo'].entries = OrderedDict()
for entry in entries:
    if not include_species(entry.item):
        continue
    ark_thermo_database.libraries['thermo'].entries[entry.label] = entry

In [ ]:
# save the results
ark_thermo_database.save_libraries('thermo')

# Add kinetics

In [4]:
# first, get valid kinetics from old workflow
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
kinetics_libs = glob.glob(os.path.join(DFT_DIR, 'kinetics', 'reaction*', 'arkane', 'RMG_libraries'))

# Load the Arkane kinetics
entries = []
for i, lib_path in enumerate(kinetics_libs):
    
    # only use kinetics that were verified with vibrational check
    vib_path = os.path.join(os.path.dirname(lib_path), 'vibrational_analysis_check.txt')
    if not os.path.exists(vib_path):
        continue
    with open(vib_path, 'r') as f:
        check = f.read()
        if check != 'True':
            continue
    
    matches = re.search('reaction_([0-9]{4})', lib_path)
    reaction_index = int(matches[1])
    ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
    ark_kinetics_database.load_libraries(lib_path)
    
    # TODO fix bug related to load_libraries not getting the actual name
    for key in ark_kinetics_database.libraries[''].entries.keys():
        entry = ark_kinetics_database.libraries[''].entries[key]
        entry.index = reaction_index
        entries.append(entry)
        print(f'Adding\t{entry.index}\t{entry}')

Adding	127	CH2O(9) + OH(15) <=> H2O(8) + HCO(19)
Adding	180	CH(3) + H2(13) <=> CH2(23) + H(14)
Adding	247	butane(1) + OH(15) <=> H2O(8) + SC4H9(183)
Adding	288	butane(1) + OH(15) <=> H2O(8) + PC4H9(182)
Adding	290	butane(1) + HO2(16) <=> H2O2(17) + PC4H9(182)
Adding	294	CH2O(9) + PC4H9(182) <=> butane(1) + HCO(19)
Adding	324	S(229) <=> S(225)
Adding	325	S(225) <=> S(244)
Adding	326	S(225) <=> S(238)
Adding	370	CH3CHO(35) + SC4H9(183) <=> butane(1) + CH2CHO(21)
Adding	371	CH3CHO(35) + PC4H9(182) <=> butane(1) + CH2CHO(21)
Adding	585	CH2(23) + C3H6(12) <=> CH3(18) + C3H5-A(94)
Adding	591	CH3CHO(35) + C3H5-A(94) <=> C3H6(12) + CH3CO(20)
Adding	650	HO2(16) + C4H8(748) <=> O2(2) + SC4H9(183)
Adding	655	HO2(16) + C4H8(748) <=> O2(2) + PC4H9(182)
Adding	739	C4H8(188) + CH2CHO(21) <=> CH3CHO(35) + C4H7(190)
Adding	748	C4H8(189) + HCO(19) <=> CH2O(9) + C4H7(190)
Adding	749	C4H8(188) + HCO(19) <=> CH2O(9) + C4H7(190)
Adding	754	C4H8(189) + CH2(23) <=> CH3(18) + C4H7(190)
Adding	894	C3H6(12) + S(

In [5]:
# entries

In [6]:
# next, get kinetics from new workflow (overwrite old if repeates)
DFT_DIR = "/work/westgroup/harris.se/autoscience/reaction_calculator/dft/"
kinetics_libs = glob.glob(os.path.join(DFT_DIR, 'kinetics', 'reaction*', 'arkane', 'RMG_libraries'))

# Load the Arkane kinetics
for i, lib_path in enumerate(kinetics_libs):

    matches = re.search('reaction_([0-9]{4})', lib_path)
    reaction_index = int(matches[1])
    ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
    ark_kinetics_database.load_libraries(lib_path)
    print(lib_path)
    # TODO fix bug related to load_libraries not getting the actual name
    existing_indices = [e.index for e in entries]
    for key in ark_kinetics_database.libraries[''].entries.keys():
        entry = ark_kinetics_database.libraries[''].entries[key]
        entry.index = reaction_index
        
        # overwrite existing
        try:
            place = existing_indices.index(reaction_index)
            entries[place] = entry
            print(f'Overwriting existing entry {reaction_index}')
        except ValueError:
            entries.append(entry)
            print(f'Adding new entry {reaction_index}')
            


/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0213/arkane/RMG_libraries
Adding new entry 213
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0288/arkane/RMG_libraries
Overwriting existing entry 288
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0370/arkane/RMG_libraries
Overwriting existing entry 370
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0410/arkane/RMG_libraries
Adding new entry 410
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0419/arkane/RMG_libraries
Adding new entry 419
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0420/arkane/RMG_libraries
Adding new entry 420
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0422/arkane/RMG_libraries
Adding new entry 422
/work/westgroup/harris.se/autoscience/reaction_calculator/dft/kinetics/reaction_0459/arkane

In [7]:
entries

[<Entry index=127 label="CH2O(9) + OH(15) <=> H2O(8) + HCO(19)">,
 <Entry index=180 label="CH(3) + H2(13) <=> CH2(23) + H(14)">,
 <Entry index=247 label="butane(1) + OH(15) <=> H2O(8) + SC4H9(183)">,
 <Entry index=288 label="OH(15) + butane(1) <=> H2O(8) + PC4H9(182)">,
 <Entry index=290 label="butane(1) + HO2(16) <=> H2O2(17) + PC4H9(182)">,
 <Entry index=294 label="CH2O(9) + PC4H9(182) <=> butane(1) + HCO(19)">,
 <Entry index=324 label="S(229) <=> S(225)">,
 <Entry index=325 label="S(225) <=> S(244)">,
 <Entry index=326 label="S(225) <=> S(238)">,
 <Entry index=370 label="CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)">,
 <Entry index=371 label="CH3CHO(35) + PC4H9(182) <=> butane(1) + CH2CHO(21)">,
 <Entry index=585 label="CH2(23) + C3H6(12) <=> CH3(18) + C3H5-A(94)">,
 <Entry index=591 label="CH3CHO(35) + C3H5-A(94) <=> C3H6(12) + CH3CO(20)">,
 <Entry index=650 label="HO2(16) + C4H8(748) <=> O2(2) + SC4H9(183)">,
 <Entry index=655 label="HO2(16) + C4H8(748) <=> O2(2) + PC4H9(182

In [9]:
top_50_rmg = [931,213,915,2434,748,793,2248,324,422,99,419,1665,1736,749,212,724,1814,714,1287,459,800,102,2311,1111,723,1761,1683,577,794,1110,728,1288,
2317,410,726,1103,2437,1766,578,1721,370,843,1658,833,1685,1522,795,1098,371,586,2321,420,
]

In [10]:
def include_reaction(rxn):
#     for j in range(0, len(reaction_list)):
    for j in top_50_rmg:
        if reaction_list[j].is_isomorphic(rxn):
#             print(rxn)
            if rmg_rxn_uncertainty[j] > 3.0:
                return True
    return False
#     print(rxn)

In [11]:
count = 0
for k in range(len(entries)):
    if include_reaction(entries[k].item):
        count += 1
        print(k, include_reaction(entries[k].item))
        
print(count)

6 True
9 True
10 True
16 True
17 True
29 True
30 True
31 True
32 True
33 True
34 True
37 True
38 True
41 True
42 True
46 True
48 True
49 True
50 True
51 True
52 True
54 True
56 True
57 True
60 True
63 True
64 True
66 True
67 True
68 True
69 True
71 True
32


In [12]:
# compile it all into a single database and a single library which I'll call harris_butane
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.libraries['kinetics'] = rmgpy.data.kinetics.KineticsLibrary()
ark_kinetics_database.libraries['kinetics'].label = 'harris_butane'
ark_kinetics_database.libraries['kinetics'].name = 'harris_butane'
ark_kinetics_database.libraries['kinetics'].entries = OrderedDict()
for entry in entries:
    if not include_reaction(entry.item):
        continue
#     if entry.index not in top_50:
#         print(f'skipping non-top-50 {entry.index}')
#         continue
    ark_kinetics_database.libraries['kinetics'].entries[entry.label] = entry

In [13]:
# save the results
# output_path = os.path.join(DFT_DIR, 'kinetics')
# ark_kinetics_database.save_libraries(output_path, reindex=False)
ark_kinetics_database.save_libraries('harris_kinetics', reindex=False)

In [14]:
# load it again just to check?

In [15]:
print(len(ark_kinetics_database.libraries['kinetics'].entries), 'saved')

32 saved


In [ ]:
for entry in ark_kinetics_database.libraries['kinetics'].entries:
    print(ark_kinetics_database.libraries['kinetics'].entries[entry].index, 
          ark_kinetics_database.libraries['kinetics'].entries[entry].label)

In [ ]:
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin, dictionary_path=species_dict, transport_path=transport, use_chemkin_names=True)


In [ ]:
for sp in species_list:
    print('species(')
    print(f'\tlabel = "{str(sp)}",')
    print(f'\treactive = True,')
    print(f'\tstructure = SMILES("{sp.smiles}"),')
    print(')')
    print()
#     print(sp)